# EFM Paper 2 Simulation: Eholokon Dynamics

This notebook simulates eholokon dynamics for the paper 'The Ehokolo Fluxon Model: A Foundation for Physics from Eholokon Dynamics.' We model a scalar field \(\phi\) with EM coupling on a 200³ grid (2 nm, atomic scales) to derive particle properties (mass, spin, charge) and validate electromagnetic interactions in the S=T state. Weak, strong, and gravity forces are approximated via relaxation, clustering, and density gradients. A 1000³ grid is used for cosmological validation where appropriate, ensuring efficient resource use on Google Colab Pro+.

## Objectives
- Simulate a 200³ grid for EM dynamics and particle properties in S=T.
- Approximate weak (relaxation), strong (clustering), and gravity (density) forces.
- Validate against NIST atomic spectra (hydrogen emission lines).
- Test a 1000³ grid for cosmological scales, optimizing memory usage.
- Provide transparent code for peer validation.

## Hardware
- **GPU**: NVIDIA A100 (40GB VRAM)
- **System RAM**: ~80GB
- **Environment**: Google Colab Pro+

## Setup Instructions
1. Go to `Runtime` > `Change runtime type` > Select `A100 GPU`.
2. Run `!nvidia-smi` to verify GPU.
3. Execute all cells sequentially.

In [ ]:
# Clear GPU memory
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
import gc
gc.collect()

# Install dependencies
!pip install --quiet torch numpy matplotlib tqdm psutil scipy scikit-learn
!nvidia-smi

# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import psutil
import time
from datetime import datetime
from IPython.display import Image, display
from scipy.fft import fftn, fftfreq
from scipy.optimize import curve_fit
from sklearn.cluster import DBSCAN
import os

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if device.type == 'cuda':
    try:
        print(f'GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
    except:
        print('Error: No GPU detected.')
print(f'System RAM Total: {psutil.virtual_memory().total / 1e9:.2f} GB')
print(f'System RAM Available: {psutil.virtual_memory().available / 1e9:.2f} GB')

# Mount Google Drive
from google.colab import drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f'Error mounting Google Drive: {e}')
    raise

# Define directories
checkpoint_path = '/content/drive/MyDrive/EFM_checkpoints_paper2/'
data_path = '/content/drive/MyDrive/EFM_data_paper2/'
for path in [checkpoint_path, data_path]:
    try:
        os.makedirs(path, exist_ok=True)
    except Exception as e:
        print(f'Error creating directory {path}: {e}')
        raise

# NIST atomic spectra (hydrogen lines, wavelengths in meters)
nist_spectra = {'H_alpha': 656.3e-9, 'H_beta': 486.1e-9, 'H_gamma': 434.0e-9}

## Simulation Setup

- **Grid Size**: 200³ (atomic, 2 nm); 1000³ (cosmological, 10 Mpc, where appropriate)
- **Box Size**: 2e-9 m (atomic); 10 Mpc (cosmological)
- **Time Step**: 1e-15 s (atomic); 0.0005 units (cosmological, TBD)
- **Steps**: 5000 (atomic); 10000 (cosmological)
- **Initial Conditions**: Gaussian pulse (\(\phi = 0.3 e^{-R^2 / (2 \sigma^2)}\), \(\sigma=0.1 \, \text{nm}\)) for S/T (atomic); two sech profiles for S/T (cosmological). T/S as gradient of S/T, S=T as \(\phi - \phi_{\text{TS}}\). Maps to \(n'=1\) (S=T, resonant).
- **Boundary Conditions**: Absorbing (damping factor=0.05, width=5% of grid)
- **Parameters**: \(m=0.0005\), \(g=3.3\), \(\eta=0.012\), \(q=0.01\), \(\delta=0.06\), \(\gamma=0.0225\), \(k=0.01\) (derived via stability analysis to match NIST spectra).
- **Notes**: Atomic simulations validate EM dynamics; cosmological tests use a simplified NLKG for clustering and \(H_0\).

The simulation models \(\phi\) with EM coupling for particle properties and Maxwell’s equations, approximating other forces via field dynamics.

In [ ]:
# Parameters
grid_size = 200  # Set to 1000 for cosmological runs
L = 2e-9  # 2 nm (atomic); 10e6 for cosmological (10 Mpc)
dx = L / grid_size
dt = 1e-15  # 0.0005 for cosmological
T = 5000   # 10000 for cosmological
m = 0.0005  # Derived via stability analysis
g = 3.3     # Nonlinear coupling for optical resonance
eta = 0.012 # Singularity prevention
q = 0.01    # EM coupling strength
delta = 0.06
gamma = 0.0225
c = 3e8
k = 0.01  # Calibrated to electron mass (TBD)

# Initialize complex field
torch.set_default_dtype(torch.complex64)
x = torch.linspace(-L/2, L/2, grid_size, device=device, dtype=torch.float32)
X, Y, Z = torch.meshgrid(x, x, x, indexing='ij')
R = torch.sqrt(X**2 + Y**2 + Z**2)
sigma = 1e-10  # 1.0 for cosmological
phi = 0.3 * torch.exp(-R**2 / (2 * sigma**2)) * torch.ones((grid_size, grid_size, grid_size), device=device, dtype=torch.complex64)
phi_dot = torch.zeros_like(phi, dtype=torch.complex64)
A_mu = torch.zeros_like(phi, dtype=torch.float32)  # Scalar potential
del X, Y, Z, R
torch.cuda.empty_cache()

# Potential
def potential(phi):
    phi_abs_sq = torch.abs(phi)**2
    return (m**2 / 2) * phi_abs_sq + (g / 4) * phi_abs_sq**2 + (eta / 6) * phi_abs_sq**3

# NLKG derivative
def nlkg_derivative(phi, phi_dot, A_mu):
    with torch.no_grad():
        # Optimized Laplacian
        kernel = torch.tensor([[[0, 0, 0], [0, 1, 0], [0, 0, 0]],
                               [[0, 1, 0], [1, -6, 1], [0, 1, 0]],
                               [[0, 0, 0], [0, 1, 0], [0, 0, 0]]], dtype=torch.float32, device=device)
        kernel = kernel / dx**2
        laplacian = torch.nn.functional.conv3d(phi.unsqueeze(0).unsqueeze(0).real, kernel.unsqueeze(0).unsqueeze(0), padding=1).squeeze()

        # Absorbing boundaries
        boundary_width = int(0.05 * grid_size)
        damping_factor = 0.05
        mask = torch.ones_like(phi, dtype=torch.float32)
        for dim in range(3):
            indices = torch.arange(grid_size, device=device)
            damping = torch.ones(grid_size, device=device, dtype=torch.float32)
            damping[:boundary_width] = damping_factor + (1 - damping_factor) * indices[:boundary_width] / boundary_width
            damping[-boundary_width:] = damping_factor + (1 - damping_factor) * (grid_size - 1 - indices[-boundary_width:]) / boundary_width
            if dim == 0:
                mask = damping[:, None, None] * mask
            elif dim == 1:
                mask = damping[None, :, None] * mask
            else:
                mask = damping[None, None, :] * mask
        phi_damped = phi * mask
        phi_dot_damped = phi_dot * mask

        # Covariant derivative
        D_t_phi = phi_dot - 1j * q * A_mu * phi
        grad_phi = torch.stack(torch.gradient(phi_damped.real, spacing=dx, dim=[0, 1, 2]))
        D2_phi = c**2 * laplacian - 1j * q * torch.stack(torch.gradient(A_mu, spacing=dx, dim=[0, 1, 2]))

        # NLKG terms
        phi_abs_sq = torch.abs(phi_damped)**2
        dV_dphi = (m**2 * phi_damped + g * phi_abs_sq * phi_damped + eta * phi_abs_sq**2 * phi_damped)
        dissipation = delta * (torch.abs(D_t_phi)**2) * phi_damped
        reciprocity = gamma * phi_damped
        phi_ddot = -D2_phi - dV_dphi - dissipation + reciprocity

        # Maxwell's equation (simplified)
        J_mu = 1j * q * (torch.conj(phi_damped) * D_t_phi - phi_damped * torch.conj(D_t_phi))
        lap_A = torch.nn.functional.conv3d(A_mu.unsqueeze(0).unsqueeze(0), kernel.unsqueeze(0).unsqueeze(0), padding=1).squeeze()
        A_mu_new = lap_A + torch.real(J_mu)

        return phi_dot_damped, phi_ddot, A_mu_new

# RK4 integrator
def update_phi(phi, phi_dot, A_mu, dt):
    with torch.no_grad():
        k1_v, k1_a, k1_A = nlkg_derivative(phi, phi_dot, A_mu)
        k2_v, k2_a, k2_A = nlkg_derivative(phi + 0.5 * dt * k1_v, phi_dot + 0.5 * dt * k1_a, A_mu + 0.5 * dt * k1_A)
        k3_v, k3_a, k3_A = nlkg_derivative(phi + 0.5 * dt * k2_v, phi_dot + 0.5 * dt * k2_a, A_mu + 0.5 * dt * k2_A)
        k4_v, k4_a, k4_A = nlkg_derivative(phi + dt * k3_v, phi_dot + dt * k3_a, A_mu + dt * k3_A)
        phi_new = phi + (dt / 6.0) * (k1_v + 2 * k2_v + 2 * k3_v + k4_v)
        phi_dot_new = phi_dot + (dt / 6.0) * (k1_a + 2 * k2_a + 2 * k3_a + k4_a)
        A_mu_new = A_mu + (dt / 6.0) * (k1_A + 2 * k2_A + 2 * k3_A + k4_A)
        del k1_v, k1_a, k1_A, k2_v, k2_a, k2_A, k3_v, k3_a, k3_A, k4_v, k4_a, k4_A
        torch.cuda.empty_cache()
        return phi_new, phi_dot_new, A_mu_new

# Energy calculation
def compute_energy(phi, phi_dot):
    with torch.no_grad():
        grad_phi = torch.stack(torch.gradient(phi.real, spacing=dx, dim=[0, 1, 2]))
        kinetic = 0.5 * torch.abs(phi_dot)**2
        gradient = 0.5 * torch.sum(grad_phi**2, dim=0)
        potential_energy = potential(phi)
        total = torch.sum(kinetic + gradient + potential_energy) * dx**3
        return total.item(), torch.sum(kinetic).item() * dx**3, torch.sum(gradient).item() * dx**3, torch.sum(potential_energy).item() * dx**3

# Mass
def compute_mass(phi, dx, k=0.01):
    return k * torch.sum(torch.abs(phi)**2) * dx**3

# Spin
def compute_spin(phi, dx):
    grad_phi = torch.stack(torch.gradient(phi.real, spacing=dx, dim=[0, 1, 2]))
    curl = grad_phi[1] * grad_phi[2] - grad_phi[2] * grad_phi[1]  # x-component
    return torch.sum(torch.abs(curl)).item() * dx**3

# Charge
def compute_charge(phi, phi_dot, A_mu, dx, q=0.01):
    D_t_phi = phi_dot - 1j * q * A_mu * phi
    J_mu = 1j * q * (torch.conj(phi) * D_t_phi - phi * torch.conj(D_t_phi))
    return torch.sum(torch.real(J_mu)).item() * dx**3

# Frequency spectrum
def compute_frequency_spectrum(phi, dt, N_t):
    phi_center = phi[grid_size//2, grid_size//2, grid_size//2].cpu().numpy()
    fft_result = np.fft.fft(phi_center)
    freqs = np.fft.fftfreq(N_t, dt)
    return freqs[:N_t//2], np.abs(fft_result)[:N_t//2]

# Weak force relaxation
def compute_relaxation(energy_history):
    if len(energy_history) < 10:
        return 0
    t = np.arange(len(energy_history))
    def exp_decay(t, A, tau):
        return A * np.exp(-t / tau)
    popt, _ = curve_fit(exp_decay, t, energy_history, p0=[energy_history[0], 1000])
    return popt[1]  # Decay time

# Strong force binding (DBSCAN)
def compute_binding(phi, threshold=0.5):
    rho = torch.abs(phi)**2
    coords = np.where(rho.cpu().numpy() > threshold)
    if len(coords[0]) < 2:
        return 0
    points = np.vstack(coords).T
    db = DBSCAN(eps=dx*2, min_samples=5).fit(points)
    return len(np.unique(db.labels_)) - (1 if -1 in db.labels_ else 0)

## Parameter Derivation

Parameters (\(m=0.0005\), \(g=3.3\), \(\eta=0.012\), \(q=0.01\), \(\delta=0.06\), \(\gamma=0.0225\), \(k=0.01\)) are derived via stability analysis of the NLKG equation for \(n'=1\) (S=T, resonant state). We solve for steady-state \(\phi_{n'} = \sqrt{\rho_{n'} / k}\), with \(\rho_{n'} = \rho_{\text{ref}} / n'\), \(\rho_{\text{ref}} = 1.5\), \(k=0.01\), ensuring frequency peaks at ~5×10¹⁴ Hz (NIST hydrogen lines). The analysis is preliminary and will be refined in future runs.

## Simulation Loop

The loop updates \(\phi\) and \(A_\mu\), tracks energy, and computes particle properties (mass, spin, charge) and force analogues (weak: relaxation, strong: clustering, gravity: density). Results are validated against NIST spectra. A 1000³ grid is used for cosmological tests with adjusted parameters.

In [ ]:
energy_history = []
mass_history = []
spin_history = []
charge_history = []
binding_history = []
phi_center_time = []
start_time = time.time()

pbar = tqdm(range(T), desc='Simulation Progress')
for t in pbar:
    phi, phi_dot, A_mu = update_phi(phi, phi_dot, A_mu, dt)
    grad_ST = torch.stack(torch.gradient(phi.real, spacing=dx, dim=[0, 1, 2]))
    phi_TS = -torch.sqrt(grad_ST[0]**2 + grad_ST[1]**2 + grad_ST[2]**2).to(dtype=torch.complex64)
    phi_S_eq_T = phi - phi_TS

    total_energy, kinetic, gradient, pot_energy = compute_energy(phi, phi_dot)
    energy_history.append(total_energy)
    mass_history.append(compute_mass(phi, dx).item())
    spin_history.append(compute_spin(phi.real, dx))
    charge_history.append(compute_charge(phi, phi_dot, A_mu, dx))
    binding_history.append(compute_binding(phi))
    phi_center_time.append(phi[grid_size//2, grid_size//2, grid_size//2].item())

    # NIST validation
    if t % 1000 == 0 or t == T - 1:
        freqs, fft_magnitude = compute_frequency_spectrum(torch.tensor(phi_center_time), dt, T)
        peak_freq = freqs[np.argmax(fft_magnitude)]
        peak_wavelength = c / peak_freq
        print(f'Step {t}: Peak frequency {peak_freq:.2e} Hz, Wavelength {peak_wavelength:.2e} m')
        for line, wavelength in nist_spectra.items():
            error = abs(peak_wavelength - wavelength) / wavelength * 100
            print(f'  {line}: NIST {wavelength:.2e} m, Error {error:.2f}%')

    # Save and visualize
    if t % 1000 == 0 or t == T - 1:
        try:
            torch.save({'step': t, 'phi': phi, 'phi_dot': phi_dot, 'A_mu': A_mu},
                       f'{checkpoint_path}checkpoint_{t}.pt')
            print(f'Checkpoint saved at step {t}')
        except Exception as e:
            print(f'Error saving checkpoint: {e}')

        phi_ST_np = phi.cpu().numpy().real
        plt.figure(figsize=(10, 8))
        ax = plt.axes(projection='3d')
        x = np.linspace(-L/2, L/2, grid_size)
        X, Y, Z = np.meshgrid(x, x, x)
        mask = phi_ST_np > 0.1
        ax.scatter(X[mask], Y[mask], Z[mask], c=phi_ST_np[mask], cmap='viridis', s=1)
        ax.set_title(f'S/T Field at Step {t}')
        plt.savefig(f'{data_path}fields_step_{t}.png')
        plt.close()

    vram_used = torch.cuda.memory_allocated() / 1e9 if device.type == 'cuda' else 0
    ram_used = psutil.virtual_memory().used / 1e9
    if t % 100 == 0:
        pbar.set_postfix({'VRAM': f'{vram_used:.2f}GB', 'RAM': f'{ram_used:.2f}GB'})
    if vram_used > 32 or ram_used > 64:
        print(f'Warning: Resource usage high at step {t}')
        break

print(f'Simulation completed in {time.time() - start_time:.2f} seconds')

## Analysis and Visualization

Analyze energy conservation, particle properties (mass, spin, charge), frequency spectrum (NIST validation), and force analogues (weak: decay time, strong: clusters, gravity: density gradients). Results are saved for peer validation.

In [ ]:
plt.figure(figsize=(12, 8))
energy_history_clean = [e for e in energy_history if np.isfinite(e)]
plt.semilogy(energy_history_clean, label='Total Energy')
plt.xlabel('Step')
plt.ylabel('Energy (log scale)')
plt.title('Energy Conservation')
plt.grid(True)
plt.legend()
plt.savefig(f'{data_path}energy_total.png')
plt.close()
display(Image(filename=f'{data_path}energy_total.png'))

# Particle properties
print(f'Final Mass: {mass_history[-1]:.4e} (calibrated TBD)')
print(f'Final Spin: {spin_history[-1]:.4e} (calibrated TBD)')
print(f'Final Charge: {charge_history[-1]:.4e} (calibrated TBD)')
print(f'Final Binding: {binding_history[-1]} clusters')

# NIST validation
freqs, fft_magnitude = compute_frequency_spectrum(torch.tensor(phi_center_time), dt, T)
peak_freq = freqs[np.argmax(fft_magnitude)]
peak_wavelength = c / peak_freq
print(f'Final Peak Frequency: {peak_freq:.2e} Hz, Wavelength: {peak_wavelength:.2e} m')
for line, wavelength in nist_spectra.items():
    error = abs(peak_wavelength - wavelength) / wavelength * 100
    print(f'{line}: NIST {wavelength:.2e} m, Error {error:.2f}%')

# Weak force decay
t = np.arange(len(energy_history_clean))
def exp_decay(t, A, tau):
    return A * np.exp(-t / tau)
popt, _ = curve_fit(exp_decay, t, energy_history_clean, p0=[energy_history_clean[0], 1000])
print(f'Weak Force Decay Time: {popt[1]:.2f} steps')

# Save data
try:
    np.save(f'{data_path}mass_history.npy', np.array(mass_history))
    np.save(f'{data_path}spin_history.npy', np.array(spin_history))
    np.save(f'{data_path}charge_history.npy', np.array(charge_history))
    np.save(f'{data_path}binding_history.npy', np.array(binding_history))
    np.save(f'{data_path}frequency_spectrum_freqs.npy', freqs)
    np.save(f'{data_path}frequency_spectrum_values.npy', fft_magnitude)
except Exception as e:
    print(f'Error saving data: {e}')